In [32]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR, LinearSVR
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy

In [33]:
#Daten einlesen
input_dir = './Data/Preparation/merged_typologien.xlsx'
df = pd.read_excel(input_dir)

In [34]:
#rows mit ganzem Kanton luzern löschen
indexNames = df[df['Gemeinde'] == 'Kanton Luzern'].index
indexNames
df.drop(indexNames , axis=0, inplace=True)
df = df.reset_index()
df.drop('index',  axis=1, inplace=True)

In [40]:
def select_typologie(df, typologie):
    typologie_df = df[df['Gemeindetypologien'] == typologie]

    #Kategrien erstellen
    typologie_df = pd.get_dummies(typologie_df, columns=['Gemeinde'])

    #manuelles splitting für die Jahre
    1991 - 2018 #training
    2019, 2020, 2021 #test

    test = typologie_df[typologie_df['Jahr'] > 2018]
    train = typologie_df[typologie_df['Jahr'] <= 2018] 

    #drop columns   
    columns_drop = ['Mittlere Wohnbevölkerung',
                    'Gemeinde ID',
                    'Ständige Wohnbevölkerung Total',
                    'Ständige Wohnbevölkerung Anteil 0-19-Jährige in %',
                    'Ständige Wohnbevölkerung Anteil 20-64-Jährige in %',
                    'Ständige Wohnbevölkerung Anteil 65-Jährige und Ältere in %',
                    'Ständige Wohnbevölkerung Ausländer-anteil in %',
                    'Gemeindetypologien']

    train = train.drop(columns_drop, axis=1)
    test = test.drop(columns_drop, axis=1)

    return typologie_df, train, test




In [36]:
df.Gemeindetypologien.unique()

array(['Agglokern', 'Land', 'Aggloguertel', 'Stadt', 'Kern'], dtype=object)

In [81]:
typologie_df, train, test = select_typologie(df, 'Land')


In [77]:
typologie_df.head()

,Gemeinde ID,Mittlere Wohnbevölkerung,Ständige Wohnbevölkerung Total,Ständige Wohnbevölkerung Ausländer-anteil in %,Ständige Wohnbevölkerung Bevölkerungs-dichte1 in Pers./km2,Ständige Wohnbevölkerung Anteil 0-19-Jährige in %,Ständige Wohnbevölkerung Anteil 20-64-Jährige in %,Ständige Wohnbevölkerung Anteil 65-Jährige und Ältere in %,Jahr,Gemeindetypologien,Gemeinde_Oberkirch,Gemeinde_Schenkon,Gemeinde_Sursee
50,1095.0,2337,2328,8.5,212.5,29.3,60.3,10.4,1991,Kern,1,0,0
61,1099.0,1734,1753,5.5,227.4,33.1,60.5,6.4,1991,Kern,0,1,0
68,1103.0,7992,7938,15.9,1311.9,25.1,61.6,13.3,1991,Kern,0,0,1
130,1095.0,2355,2340,9.8,213.6,29.4,60.3,10.3,1992,Kern,1,0,0
141,1099.0,1758,1751,4.9,227.1,33.1,60.7,6.3,1992,Kern,0,1,0


In [54]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error
import matplotlib.pyplot as plt

In [82]:
gemeinden = typologie_df.columns.tolist()
gemeinden = gemeinden[10:]
gemeinden[0]

'Gemeinde_Aesch'

# Polynomial Regression

In [83]:
X_train =  train.drop('Ständige Wohnbevölkerung Bevölkerungs-dichte1 in Pers./km2', axis=1)
y_train = train['Ständige Wohnbevölkerung Bevölkerungs-dichte1 in Pers./km2']
X_test =  test.drop('Ständige Wohnbevölkerung Bevölkerungs-dichte1 in Pers./km2', axis=1)
y_test = test['Ständige Wohnbevölkerung Bevölkerungs-dichte1 in Pers./km2']

from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=3, include_bias=True)

x_train_poly_features = poly.fit_transform(X_train)
x_test_poly_features = poly.transform(X_test)

from sklearn.linear_model import LinearRegression
poly_reg_model = LinearRegression()
poly_reg_model.fit(x_train_poly_features, y_train)
poly_y_predicted = poly_reg_model.predict(x_test_poly_features)

R2 = r2_score(y_test, poly_y_predicted)
mse = mean_squared_error(y_test, poly_y_predicted)
mape = mean_absolute_percentage_error(y_test, poly_y_predicted)
print("R2:", R2)
print("mse:", mse)
print("mape:", mape)

R2: 0.9922149170642041
mse: 271.8826150708649
mape: 0.07982491931726177


In [ ]:
#Gemeinden ploten
for i in range(0,len(gemeinden)):
    ge = typologie_df[typologie_df[gemeinden[i]] == 1]
    columns_drop_2 = ['Mittlere Wohnbevölkerung',
                    'Gemeinde ID',
                    'Ständige Wohnbevölkerung Total',
                    'Ständige Wohnbevölkerung Anteil 0-19-Jährige in %',
                    'Ständige Wohnbevölkerung Anteil 20-64-Jährige in %',
                    'Ständige Wohnbevölkerung Anteil 65-Jährige und Ältere in %',
                    'Ständige Wohnbevölkerung Ausländer-anteil in %',
                    'Ständige Wohnbevölkerung Bevölkerungs-dichte1 in Pers./km2',
                    'Gemeindetypologien']

    ge_x = ge.drop(columns_drop_2, axis=1)
    ge_y = ge['Ständige Wohnbevölkerung Bevölkerungs-dichte1 in Pers./km2']

    #normalisieren
    ge_x = poly.transform(ge_x)
    #vorhersage
    ge_pred = poly_reg_model.predict(ge_x)
    

    Jahre=ge.Jahr.tolist()

    plt.figure()
    plt.scatter(Jahre,ge['Ständige Wohnbevölkerung Bevölkerungs-dichte1 in Pers./km2'])
    plt.plot(Jahre, ge_pred, color="black")
    plt.scatter(Jahre[-4:], ge_pred[-4:], color='red')
    plt.savefig('../Plots_Model/nach_Typologien/Land/' + gemeinden[i] + '.png')
    plt.show()
    

Neue Plots

In [ ]:
#Gemeinden ploten
for i in range(0,len(gemeinden)):
    ge = typologie_df[typologie_df[gemeinden[i]] == 1]
    columns_drop_2 = ['Mittlere Wohnbevölkerung',
                    'Gemeinde ID',
                    'Ständige Wohnbevölkerung Total',
                    'Ständige Wohnbevölkerung Anteil 0-19-Jährige in %',
                    'Ständige Wohnbevölkerung Anteil 20-64-Jährige in %',
                    'Ständige Wohnbevölkerung Anteil 65-Jährige und Ältere in %',
                    'Ständige Wohnbevölkerung Ausländer-anteil in %',
                    'Ständige Wohnbevölkerung Bevölkerungs-dichte1 in Pers./km2',
                    'Gemeindetypologien']

    ge_x = ge.drop(columns_drop_2, axis=1)
    ge_y = ge['Ständige Wohnbevölkerung Bevölkerungs-dichte1 in Pers./km2']

    #normalisieren
    ge_x = poly.transform(ge_x)
    #vorhersage
    ge_pred = poly_reg_model.predict(ge_x)
    

    Jahre=ge.Jahr.tolist()

    
    plt.figure(figsize=(8, 12))
    #plt.plot(Jahre,ge['Ständige Wohnbevölkerung Bevölkerungs-dichte1 in Pers./km2'], color="black", linewidth=2)
    #plt.plot(Jahre[-4:],ge['Ständige Wohnbevölkerung Bevölkerungs-dichte1 in Pers./km2'][-4:], color="blue", linewidth=2)
    #plt.plot(Jahre, ge_pred, color="black")
    #plt.plot(Jahre[-4:], ge_pred[-4:], color='red', linewidth=3)
    plt.scatter(Jahre,ge['Ständige Wohnbevölkerung Bevölkerungs-dichte1 in Pers./km2'], color="black", s=70)
    plt.scatter(Jahre[-4:],ge['Ständige Wohnbevölkerung Bevölkerungs-dichte1 in Pers./km2'][-4:], color="lightseagreen", s=70)
    plt.plot(Jahre, ge_pred, color="red", linewidth=2)
    #plt.scatter(Jahre[-4:], ge_pred[-4:], color='red', s=40)
    
    ax = plt.gca()
    for spine in ax.spines.values():
        spine.set_linewidth(0)
    plt.ylim(0, 2300) 
    plt.xticks(fontsize=14)  # set the font size of the x-axis tick labels to 14
    plt.yticks(fontsize=14)

    plt.savefig('../Plots_Model/nach_Typologien/New_plots/kern/' + gemeinden[i] + '.png', transparent=True)

    plt.show()
    